# Hyper-optimalizace Parametru Modelu

V tomto textu se podivame na nekolik moznosti, jak optimalizovat vykonnost modelu na datech. 
Proc vlastne delame hyperoptimalizaci parametru? Protoze defaultni parametry nemusi byt vzdy vhodny. Nebo jsou suboptimalni pro vykon modelu. 
Kdyz mame data a rozhodli jsme se na ne pouzit nejaky model, dejme tomu regressni model s regularizaci, tak vnitrni parametry modelu jsou koeficienty, ktery nam vypadnou, kdyz model budeme trenovat na datech. 

Nicmene, jak zvolit treba regularizacni parametr $\alpha$, ktery nastavuje vůli regularizace? 
A co kdyz chceme pouzit L1 i L2 regularizaci? 
Jake hodnoty pro parametry $\alpha$ a $\beta$ mame zvolit? 
Tyhle dva parametry definuji celou rodinu modelů pro data data a samozrejme, jejich nastavenim se meni i vnitrni parametry modelu, i.e. koeficienty u promennych. 

Hyper-parametry se stejne jako vnitrni parametry modelu trenuji. A protoze se na soustavu $\textit{data-model}$ divame jako na jeden objekt, potrebujeme pro trenovani hyper-parametru "data". Toho se docili napriklad pomoci cross-validace, ktera data rozdeli do nahodne skupiny a pro nejaka nastaveni hyper-parametru data natrenuje. 

Nicemen, i v pripade trenovani hyper-parametru muze dojit k over-fittingu a je treba otestovat, zda model funguje. Toho docilime tim, ze model vypustime na data, ktera nebyla pouzita ani pri trenovani vnitrnich parametru, ani pri trenovani hyper-parametru. 

I vlastni vyber modelu je hyper-parametr. Tj. jestli zvolit regressi, xgboost, catboost, etc. je parametr, ktery se 
musi natrenovat.

A jak takove trenovani hyper-parametru probiha?
Jsou tri mozne bezne techniky: gird search, random search a bayesian optimization. Grid search je nejbeznejsi, ale neefektivni. Random search je efektivnejsi, ale bayesian optimization je nejefektivnejsi.

## Grid-Search

V pripade Grid Search se prochazi veskere kombinace hyper-parametru a pro kazdou kombinaci se natrenuje model.

## Random Search

V pripade Random Search se prochazi nahodne vybrany kombinace hyper-parametru a pro kazdou kombinaci se natrenuje model.

## Bayesovska Optimalizace

Bayesovská optimalizace je metoda pro hledání globálního optima black boxu nebo funkce, která je výpočetně náročná na vyhodnocení. 

- Bayesovská optimalizace kombinuje apriorní znalosti o hledané funkci a evidence získané z pozorování pro konstrukci surrogátního modelu funkce.
- Surrogátní model, obvykle Gaussovský proces, se používá k aproximaci skutečné funkce. Model se aktualizuje s každým novým pozorováním.
Akvizičních funkcí, jako je očekávané zlepšení (EI) nebo horní interval spolehlivosti (UCB), se používají k inteligentnímu výběru dalších bodů k vyhodnocení.
- Maximum akviziční funkce se obvykle hledá pomocí diskretizace nebo pomocí optimalizátoru.
- Akviziční funkce jsou maximalizovány pomocí numerické optimalizační techniky, jako je Newtonova metoda nebo kvazi-Newtonovy metody jako algoritmus Broyden–Fletcher–Goldfarb–Shanno.

![bayes](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/GpParBayesAnimationSmall.gif/532px-GpParBayesAnimationSmall.gif "bayes")

## Prakticka Ukazka

In [54]:
from hyperopt import hp, fmin, tpe, Trials, space_eval
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
samples = 20000
dummy = make_classification(n_samples=samples)

# splitting the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(dummy[0],
                                                    dummy[1],
                                                    test_size=0.2,
                                                    stratify=dummy[1],
                                                    random_state=42)

In [3]:
X_train.shape, X_test.shape

((16000, 20), (4000, 20))

### Hyperopt

Hyperopt je knihovna pro Python, která umožňuje optimalizaci hyperparametrů modelu pomocí bayesovské optimalizace.

In [29]:
space = {
    'max_depth': hp.quniform('max_depth', 10, 120, 10),
    'n_estimators': hp.quniform('n_estimators', 10, 120, 10),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 2),
    'bootstrap': hp.choice('bootstrap', [True, False])
}

In [30]:
def objective(params):
    params["max_depth"] = int(params["max_depth"])
    params["min_samples_leaf"] = int(params["min_samples_leaf"])
    params["n_estimators"] = int(params["n_estimators"])
    
    clf = RandomForestClassifier(**params)
    score = -cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return score

In [31]:
trials = Trials()

In [33]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

 90%|█████████ | 9/10 [01:51<00:15, 15.91s/trial, best loss: -0.977374047940186] 


In [34]:
best

{'bootstrap': 0,
 'criterion': 1,
 'max_depth': 110.0,
 'min_samples_leaf': 4.0,
 'n_estimators': 110.0}

In [38]:
best_params = space_eval(space, best)

In [42]:
best_params

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 110.0,
 'min_samples_leaf': 4.0,
 'n_estimators': 110.0}

In [43]:
best_params["max_depth"] = int(best_params["max_depth"])
best_params["min_samples_leaf"] = int(best_params["min_samples_leaf"])
best_params["n_estimators"] = int(best_params["n_estimators"])

In [44]:
clf = RandomForestClassifier(**best_params)

In [45]:
clf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=110, min_samples_leaf=4,
                       n_estimators=110)

In [46]:
clf.score(X_test, y_test)

0.9285

### Optuna

Optuna je knihovna pro Python, která umožňuje optimalizaci hyperparametrů modelu pomocí bayesovské optimalizace.

In [48]:
import optuna as optuna

In [49]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 10, 120, 10)
    n_estimators = trial.suggest_int('n_estimators', 10, 120, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, 2)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    
    clf = RandomForestClassifier(max_depth=max_depth,
                                 n_estimators=n_estimators,
                                 criterion=criterion,
                                 min_samples_leaf=min_samples_leaf,
                                 bootstrap=bootstrap)
    
    score = -cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return score

In [55]:
study = optuna.create_study(direction='minimize')

[I 2024-04-14 12:49:24,801] A new study created in memory with name: no-name-b19d7e14-23e2-4d29-b69d-de6f73cbc2bc


In [56]:
study.optimize(objective, n_trials=10)

[I 2024-04-14 12:49:47,325] Trial 0 finished with value: -0.9768249459403993 and parameters: {'max_depth': 40, 'n_estimators': 100, 'criterion': 'gini', 'min_samples_leaf': 7, 'bootstrap': True}. Best is trial 0 with value: -0.9768249459403993.
[I 2024-04-14 12:49:59,739] Trial 1 finished with value: -0.9755222500333401 and parameters: {'max_depth': 100, 'n_estimators': 60, 'criterion': 'gini', 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: -0.9768249459403993.
[I 2024-04-14 12:50:03,271] Trial 2 finished with value: -0.971790646555235 and parameters: {'max_depth': 20, 'n_estimators': 10, 'criterion': 'entropy', 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 0 with value: -0.9768249459403993.
[I 2024-04-14 12:50:16,433] Trial 3 finished with value: -0.9775940870084388 and parameters: {'max_depth': 90, 'n_estimators': 70, 'criterion': 'entropy', 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 3 with value: -0.9775940870084388.
[I 2024-04-14 12

In [57]:
print('Best hyperparameters:', study.best_params)

Best hyperparameters: {'max_depth': 90, 'n_estimators': 70, 'criterion': 'entropy', 'min_samples_leaf': 9, 'bootstrap': True}


In [58]:
print('Best AUC:', study.best_value)

Best AUC: -0.9775940870084388


In [59]:
study.best_trial

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[-0.9775940870084388], datetime_start=datetime.datetime(2024, 4, 14, 12, 50, 3, 271420), datetime_complete=datetime.datetime(2024, 4, 14, 12, 50, 16, 433043), params={'max_depth': 90, 'n_estimators': 70, 'criterion': 'entropy', 'min_samples_leaf': 9, 'bootstrap': True}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=120, log=False, low=10, step=10), 'n_estimators': IntDistribution(high=120, log=False, low=10, step=10), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'min_samples_leaf': IntDistribution(high=9, log=False, low=1, step=2), 'bootstrap': CategoricalDistribution(choices=(True, False))}, trial_id=3, value=None)

Reference:

- https://forecastegy.com/posts/catboost-hyperparameter-tuning-guide-with-optuna/
- https://optuna.readthedocs.io/en/stable/reference/trial.html
- https://github.com/catboost/tutorials/blob/master/hyperparameters_tuning/hyperparameters_tuning_using_optuna_and_hyperopt.ipynb
- https://neptune.ai/blog/optuna-vs-hyperopt